In [122]:
import sys
import pathlib

import joblib
import pandas as pd

# project module imports
sys.path.append("../../")  # noqa
from src.utils import (
    load_json_file,
    split_meta_and_features,
    check_feature_order,
    generate_confusion_matrix_tl,
)

## Setting up parameters and paths

In [106]:
# setting up paths and output paths
results_dir = pathlib.Path("../../results")
fs_results_dir = (results_dir / "0.feature_selection").resolve(strict=True)
data_split_dir = (results_dir / "1.data_splits/").resolve(strict=True)
jump_data_dir = pathlib.Path("../../data/JUMP_data").resolve(strict=True)
modeling_dir = pathlib.Path("../../results/2.modeling").resolve(strict=True)

# JUMP data files
jump_data_path = (jump_data_dir / "JUMP_all_plates_normalized_negcon.csv.gz").resolve(
    strict=True
)

# loading only cell injury metadata (after holdout has been applied)
cell_injury_metadata_path = (
    data_split_dir / "cell_injury_metadata_after_holdout.csv.gz"
).resolve(strict=True)

# model paths
multi_class_model_path = (modeling_dir / "multi_class_model.joblib").resolve(
    strict=True
)
shuffled_multi_class_model_path = (
    modeling_dir / "shuffled_multi_class_model.joblib"
).resolve(strict=True)

# overlapping feature space path
shared_feature_space_path = (
    fs_results_dir / "cell_injury_shared_feature_space.json"
).resolve(strict=True)

# injury codes
injury_codes_path = (data_split_dir / "injury_codes.json").resolve(strict=True)

# output paths
jump_analysis_dir = (results_dir / "3.jump_analysis").resolve()
jump_analysis_dir.mkdir(exist_ok=True)

## Loading in datasets and json files

Here we are loading the JUMP dataset along with the cell injury metadata, injury codes and the files representing the overlapping feature space.

In [107]:
# loading in JUMP dataset
jump_df = pd.read_csv(jump_data_path)

# loading in cell injury metadata only (after holdout)
cell_injury_meta_df = pd.read_csv(cell_injury_metadata_path)

# split metadata and feature columns
jump_meta, jump_feats = split_meta_and_features(jump_df, metadata_tag=True)

# loading json file that contains the shared feature
injury_codes = load_json_file(injury_codes_path)
injury_encoder = injury_codes["encoder"]
injury_decoder = injury_codes["decoder"]

# loading in shared feature space
shared_feature_space = load_json_file(shared_feature_space_path)
shared_features = shared_feature_space["features"]

# Display data
print("JUMP dataset shape", jump_df.shape)
print("Number of Meta features", len(jump_meta))
print("Number of JUMP features", len(jump_feats))
print("Number of shared features between JUMP and Cell Injury", len(shared_features))
jump_df.head()

JUMP dataset shape (19498, 5805)
Number of Meta features 13
Number of JUMP features 5792
Number of shared features between JUMP and Cell Injury 207


,Metadata_broad_sample,Metadata_Plate,Metadata_Well,Metadata_gene,Metadata_pert_type,Metadata_control_type,Metadata_target_sequence,Metadata_negcon_control_type,Cells_AreaShape_Area,Cells_AreaShape_BoundingBoxArea,...,Nuclei_Texture_Variance_RNA_3_03_256,Nuclei_Texture_Variance_RNA_5_00_256,Nuclei_Texture_Variance_RNA_5_01_256,Nuclei_Texture_Variance_RNA_5_02_256,Nuclei_Texture_Variance_RNA_5_03_256,Metadata_solvent,Metadata_InChIKey,Metadata_pert_iname,Metadata_pubchem_cid,Metadata_smiles
0,BRDN0001480888,BR00116997,A01,HIF1A,trt,NaN,TATGTGTGAATTACGTTGTG,NaN,1.365900,2.159800,...,-1.448300,-1.386300,-1.368800,-1.426400,-1.323900,NaN,NaN,NaN,NaN,NaN
1,BRDN0001483495,BR00116997,A02,CATSPER4,trt,NaN,CCGACCGTAGGACTCGTGAA,NaN,-0.577230,-0.328600,...,-0.787660,-0.782950,-0.757350,-0.770820,-0.758180,NaN,NaN,NaN,NaN,NaN
2,BRDN0001147364,BR00116997,A03,DDR2,trt,NaN,CCGTGACAAACCGAGCACTG,NaN,-0.348260,-0.077757,...,-0.612140,-0.606210,-0.582380,-0.604900,-0.569400,NaN,NaN,NaN,NaN,NaN
3,BRDN0001490272,BR00116997,A04,OPRL1,trt,NaN,AACGGGAACACCGACAACAG,NaN,-0.076636,0.302890,...,0.001638,0.005891,0.029338,-0.014926,0.055581,NaN,NaN,NaN,NaN,NaN
4,BRDN0001480510,BR00116997,A05,SLC7A11,trt,NaN,GAAGAGATTCAAGTATTACG,NaN,-0.282870,-0.064418,...,-0.431160,-0.401550,-0.374840,-0.423030,-0.343840,NaN,NaN,NaN,NaN,NaN


## Updating the JUMP Dataset by Selecting Only Shared Features

During this step, we utilize the shared feature list to update our JUMP dataset, selecting only those features that overlap.

Note that the shared feature space file maintains the same order as the feature space used during model training.

In [108]:
# update the over lapping jump df
# Augment the overlapping feature space with the metadata
shared_jump_df = jump_df[shared_features]
shared_jump_df = pd.concat([jump_df[jump_meta], shared_jump_df], axis=1)

# split the features
shared_meta, shared_feats = split_meta_and_features(shared_jump_df, metadata_tag=True)

# checking if the feature space are identical (also looks for feature space order)
assert check_feature_order(
    ref_feat_order=shared_features, input_feat_order=shared_feats
), "Feature space are not identical"

# display
print(
    "Shape of overlapping jump datadrame with overlapping features",
    shared_jump_df.shape,
)
print("Number of meta features", len(shared_meta))
print("Number of features", len(shared_feats))
shared_jump_df.head()

Shape of overlapping jump datadrame with overlapping features (19498, 220)
Number of meta features 13
Number of features 207


,Metadata_broad_sample,Metadata_Plate,Metadata_Well,Metadata_gene,Metadata_pert_type,Metadata_control_type,Metadata_target_sequence,Metadata_negcon_control_type,Metadata_solvent,Metadata_InChIKey,...,Nuclei_RadialDistribution_RadialCV_ER_2of4,Nuclei_RadialDistribution_RadialCV_ER_3of4,Nuclei_RadialDistribution_RadialCV_ER_4of4,Nuclei_RadialDistribution_RadialCV_Mito_1of4,Nuclei_RadialDistribution_RadialCV_Mito_2of4,Nuclei_RadialDistribution_RadialCV_Mito_3of4,Nuclei_RadialDistribution_RadialCV_Mito_4of4,Nuclei_RadialDistribution_RadialCV_RNA_1of4,Nuclei_RadialDistribution_RadialCV_RNA_2of4,Nuclei_RadialDistribution_RadialCV_RNA_3of4
0,BRDN0001480888,BR00116997,A01,HIF1A,trt,NaN,TATGTGTGAATTACGTTGTG,NaN,NaN,NaN,...,-3.3050,-3.5140,-4.2082,0.26573,-0.47867,-1.9730,-2.0680,-2.9914,-2.8744,-4.5757
1,BRDN0001483495,BR00116997,A02,CATSPER4,trt,NaN,CCGACCGTAGGACTCGTGAA,NaN,NaN,NaN,...,-4.3915,-4.4303,-4.6181,-1.45990,-2.19250,-2.2624,-2.4207,-2.7762,-3.4125,-4.8014
2,BRDN0001147364,BR00116997,A03,DDR2,trt,NaN,CCGTGACAAACCGAGCACTG,NaN,NaN,NaN,...,-2.1447,-1.9080,-2.6673,-1.22030,-1.54220,-1.6594,-1.7602,-2.3256,-2.4619,-3.0508
3,BRDN0001490272,BR00116997,A04,OPRL1,trt,NaN,AACGGGAACACCGACAACAG,NaN,NaN,NaN,...,-2.7450,-2.6384,-3.4326,-1.03720,-1.36520,-1.6072,-2.1697,-2.1248,-2.0682,-3.0721
4,BRDN0001480510,BR00116997,A05,SLC7A11,trt,NaN,GAAGAGATTCAAGTATTACG,NaN,NaN,NaN,...,-3.1048,-3.4037,-3.9661,-1.03410,-1.40220,-1.6420,-2.2435,-2.1156,-1.8925,-3.6533


In [109]:
# save overlapping files
shared_jump_df.to_csv(
    jump_analysis_dir / "shared_feats_jump_data.csv.gz",
    compression="gzip",
    index=False,
)

## Identifying shared treatments
Once the feature space has been narrowed down to only those features shared between both datasets, the next step is to generate a dataset containing shared treatments that are both presentin in the `cell_injury` and `JUMP` datasets. These shared compounds will then be utilized to subset the `JUMP` dataset, which will be considered as the ground truth for downstream analyses.

**Approach**:
1. **Identifying shared Compounds**: We compare the compounds present in both datasets to identify the overlapping ones.
2. **Subsetting the JUMP Dataset**: Once the overlapping compounds are identified, we subset the `JUMP` dataset to include only those compounds, forming our ground truth dataset.
3. **Save dataset**: The dataset will be saved in the `./results/3.jump_analysis`

### Identifying Overlapping Compounds
Here, we used the International Chemical Identifier (InChI) to identify chemicals shared between the JUMP dataset and the Cell Injury dataset.

In [110]:
cell_injury_InChI_keys = cell_injury_meta_df["Compound InChIKey"].unique().tolist()
jump_InChI_keys = shared_jump_df["Metadata_InChIKey"].unique().tolist()

# identify common InChI Keys
common_compounds_inchikey = list(
    set(cell_injury_InChI_keys).intersection(jump_InChI_keys)
)

# identify the compounds
overlapping_compounds_df = cell_injury_meta_df.loc[
    cell_injury_meta_df["Compound InChIKey"].isin(common_compounds_inchikey)
]

# inserting injury code
overlapping_compounds_df.insert(
    0,
    "injury_code",
    overlapping_compounds_df["injury_type"].apply(lambda name: injury_encoder[name]),
)
unique_compound_names = overlapping_compounds_df["Compound Name"].unique().tolist()
print("Identified overlapping compounds:", ", ".join(unique_compound_names))

# now create a dataframe where it contains
overlapping_compounds_df = (
    overlapping_compounds_df[
        ["injury_code", "injury_type", "Compound Name", "Compound InChIKey"]
    ]
    .drop_duplicates()
    .reset_index(drop=True)
)

# lower casing all the entries
overlapping_compounds_df["injury_type"] = overlapping_compounds_df[
    "injury_type"
].str.lower()
overlapping_compounds_df

Identified overlapping compounds: DMSO, Colchicine, Cycloheximide, Menadione


,injury_code,injury_type,Compound Name,Compound InChIKey
0,0,control,DMSO,IAZDPXIOMUYVGZ-UHFFFAOYSA-N
1,1,cytoskeletal,Colchicine,IAKHMKGGTNLKSZ-INIZCTEOSA-N
2,5,miscellaneous,Cycloheximide,YPHMISFOHDHNIV-FSZOTQKASA-N
3,6,redox,Menadione,MJVAVZPDRWSRRC-UHFFFAOYSA-N


Once the common compounds and their associated cell injury types are identified, the next step involves selecting it from the JUMP dataset to select only wells that possess the common InChI keys.

In [111]:
# selecting rows that contains the overlapping compounds
shared_treat_jump_df = shared_jump_df.loc[
    shared_jump_df["Metadata_InChIKey"].isin(common_compounds_inchikey)
]

# augment filtered JUMP data with labels
shared_treat_jump_df = pd.merge(
    overlapping_compounds_df,
    shared_treat_jump_df,
    right_on="Metadata_InChIKey",
    left_on="Compound InChIKey",
)

# shared treatment jump df
print("shape: ", shared_jump_df.shape)
shared_treat_jump_df.head()

shape:  (19498, 220)


,injury_code,injury_type,Compound Name,Compound InChIKey,Metadata_broad_sample,Metadata_Plate,Metadata_Well,Metadata_gene,Metadata_pert_type,Metadata_control_type,...,Nuclei_RadialDistribution_RadialCV_ER_2of4,Nuclei_RadialDistribution_RadialCV_ER_3of4,Nuclei_RadialDistribution_RadialCV_ER_4of4,Nuclei_RadialDistribution_RadialCV_Mito_1of4,Nuclei_RadialDistribution_RadialCV_Mito_2of4,Nuclei_RadialDistribution_RadialCV_Mito_3of4,Nuclei_RadialDistribution_RadialCV_Mito_4of4,Nuclei_RadialDistribution_RadialCV_RNA_1of4,Nuclei_RadialDistribution_RadialCV_RNA_2of4,Nuclei_RadialDistribution_RadialCV_RNA_3of4
0,0,control,DMSO,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,NaN,BR00117008,A02,NaN,control,negcon,...,-2.76110,-3.64770,-4.25560,-0.42812,-1.588900,-2.611600,-3.02970,1.2210,-0.010798,-3.57670
1,0,control,DMSO,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,NaN,BR00117008,A09,NaN,control,negcon,...,-2.12210,-2.45130,-2.09820,-1.03440,-1.546900,-1.842700,-1.49650,1.0611,0.319800,-2.29210
2,0,control,DMSO,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,NaN,BR00117008,A17,NaN,control,negcon,...,-1.01330,-0.78071,-0.97451,0.24535,0.050191,0.012229,0.43542,-1.9169,-2.295100,-0.92397
3,0,control,DMSO,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,NaN,BR00117008,B03,NaN,control,negcon,...,-1.15110,-1.01220,-1.15160,-0.27399,0.171160,0.350050,0.58907,-1.0525,-1.492700,-1.44500
4,0,control,DMSO,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,NaN,BR00117008,B14,NaN,control,negcon,...,0.27617,0.56630,0.22181,-0.55906,0.339750,0.827730,0.88401,-1.0749,-0.753400,0.20270


Now that we have identified the wells treated with overlapping treatments, we want to know the amount of wells that a specific treatment have.

In [112]:
# count number of wells and agument with injury_code injury_yype and compound name
well_counts_df = (
    shared_treat_jump_df.groupby("Metadata_InChIKey")
    # counting the numbver of wells
    .size()
    .to_frame()
    .reset_index()
    # merge based on InChIKey
    .merge(
        overlapping_compounds_df,
        left_on="Metadata_InChIKey",
        right_on="Compound InChIKey",
    )
    # remove duplicate InChIKey Column
    .drop(columns=["Compound InChIKey"])
)

# update columns
well_counts_df.columns = [
    "Metadata_InChIKey",
    "n_wells",
    "injury_code",
    "injury_type",
    "compund_name",
]
well_counts_df

,Metadata_InChIKey,n_wells,injury_code,injury_type,compund_name
0,IAKHMKGGTNLKSZ-INIZCTEOSA-N,24,1,cytoskeletal,Colchicine
1,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,1522,0,control,DMSO
2,MJVAVZPDRWSRRC-UHFFFAOYSA-N,23,6,redox,Menadione
3,YPHMISFOHDHNIV-FSZOTQKASA-N,24,5,miscellaneous,Cycloheximide


Next, we wanted to examine the distribution of treatments across plates.

In [113]:
# now lets look at the amount of wells have treatments and controls per plate
n_well_treatments = {}
for plate, df in shared_treat_jump_df.groupby("Metadata_Plate"):
    treatment_counts = {}
    for treatment, df2 in df.groupby("Metadata_InChIKey"):
        counts = df2.shape[0]
        treatment_counts[df2["Compound Name"].unique()[0]] = counts

    n_well_treatments[plate] = treatment_counts

# looking treatment distribution across each plate
plate_treatments = (
    pd.DataFrame.from_dict(n_well_treatments, orient="columns")
    .T[["DMSO", "Colchicine", "Menadione", "Cycloheximide"]]
    .fillna(0)
    .astype(int)
    .reset_index()
)
plate_treatments.columns = [
    "plate_id",
    "DMSO",
    "Colchicine",
    "Menadione",
    "Cycloheximide",
]

# display
print(
    "Number of Plates that contain overlapping treatments:", plate_treatments.shape[0]
)
plate_treatments

Number of Plates that contain overlapping treatments: 24


,plate_id,DMSO,Colchicine,Menadione,Cycloheximide
0,BR00116991,64,1,1,1
1,BR00116992,64,1,1,1
2,BR00116993,64,1,1,1
3,BR00116994,64,1,1,1
4,BR00116995,50,1,0,1
5,BR00117008,64,1,1,1
6,BR00117009,64,1,1,1
7,BR00117010,64,1,1,1
8,BR00117011,64,1,1,1
9,BR00117012,64,1,1,1


Finally we save the shared_treaments_df as a csv.gz file.

In [114]:
# save overlapping files
shared_treat_jump_df.to_csv(
    jump_analysis_dir / "shared_treatments_jump_data.csv.gz",
    compression="gzip",
    index=False,
)

## Applying JUMP dataset to Multi-Class Logistics Regression Model

In [115]:
# split the data
aligned_meta_cols, aligned_feature_cols = split_meta_and_features(shared_jump_df)

# check if the feature space are the same
X = shared_jump_df[aligned_feature_cols]


assert check_feature_order(
    ref_feat_order=shared_features, input_feat_order=X.columns.tolist()
), "Feature space are not identical"

In [116]:
# Loading in model
model = joblib.load(modeling_dir / "multi_class_model.joblib")
shuffled_model = joblib.load(modeling_dir / "shuffled_multi_class_model.joblib")

Here, we apply the JUMP dataset to the model to calculate the probabilities of each injury being present per well. These probabilities are then saved in a tidy long format suitable for plotting in R.

In [117]:
# get all injury classes
injury_classes = [injury_decoder[str(code)] for code in model.classes_.tolist()]

# prediction probabilities on both non-shuffled and shuffled models
y_proba = model.predict_proba(X)
shuffled_y_proba = shuffled_model.predict_proba(X)

# convert to pandas dataframe
y_proba_df = pd.DataFrame(y_proba)
shuffled_y_proba_df = pd.DataFrame(shuffled_y_proba)

# update column names with injury type names
y_proba_df.columns = [
    injury_codes["decoder"][str(colname)] for colname in y_proba_df.columns.tolist()
]

shuffled_y_proba_df.columns = [
    injury_codes["decoder"][str(colname)]
    for colname in shuffled_y_proba_df.columns.tolist()
]

# adding column if labels indicating if the prediction was done with a shuffled model
y_proba_df.insert(0, "shuffled_model", False)
shuffled_y_proba_df.insert(0, "shuffled_model", True)

# merge InChIKey based on index, since order is retained
# jump_df[aligned_meta_cols].merge(y_proba_df)
y_proba_df = pd.merge(
    shared_jump_df[aligned_meta_cols]["Metadata_InChIKey"].to_frame(),
    y_proba_df,
    left_index=True,
    right_index=True,
)
shuffled_y_proba_df = pd.merge(
    shared_jump_df[aligned_meta_cols]["Metadata_InChIKey"].to_frame(),
    shuffled_y_proba_df,
    left_index=True,
    right_index=True,
)

# concat all probabilities into one dataframe
all_probas_df = pd.concat([y_proba_df, shuffled_y_proba_df]).reset_index(drop=True)

# Add a column to indicate the most probable injury
# This is achieved by selecting the injury with the highest probability
all_probas_df.insert(
    2,
    "pred_injury",
    all_probas_df[injury_classes].apply(lambda row: row.idxmax(), axis=1),
)

# next is to convert the probabilities dataframe into tidy long
all_probas_df_tl = pd.melt(
    all_probas_df,
    id_vars=["Metadata_InChIKey", "shuffled_model", "pred_injury"],
    value_vars=injury_classes,
    var_name="injury_type",
    value_name="proba",
)

# save probabilities in tidy long format
all_probas_df_tl.to_csv(jump_analysis_dir / "JUMP_injury_proba.csv.gz", index=False)

print("tidy long format probability shape", all_probas_df_tl.shape)

tidy long format probability shape (584940, 5)


## Generating Confusion Matrix

In [123]:
shared_treat_meta, shared_treat_feats = split_meta_and_features(shared_treat_jump_df)
shared_X = shared_treat_jump_df[shared_treat_feats]
shared_y = shared_treat_jump_df["injury_code"]

In [124]:
jump_overlap_cm = generate_confusion_matrix_tl(
    model, shared_X, shared_y, shuffled=False, dataset_type="JUMP Overlap"
).fillna(0)
shuffled_jump_overlap_cm = generate_confusion_matrix_tl(
    shuffled_model, shared_X, shared_y, shuffled=True, dataset_type="JUMP Overlap"
).fillna(0)

In [125]:
# save confusion matrix
pd.concat([jump_overlap_cm, shuffled_jump_overlap_cm]).to_csv(
    modeling_dir / "jump_overlap_confusion_matrix.csv.gz",
    compression="gzip",
    index=False,
)